In [1]:
import numpy as np

from matplotlib import pyplot as plt
import pandas

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from SpectralClustering import SpectralClustering

* The data is a small sample from the the GSE 73072 dataset (from NCBI).
* Reactome Pathway Database to determine pathway membership and networks. 
* R package graphite to generate pathway networks with edges.

Contact nathan.mankovich@gmail.com for more information regarding the example data.

## Load the data and labels

In [2]:
ds = pandas.read_csv('../data/gse73072_4to2_9_16_subjectID_limma_test.csv', index_col = 'SampleID')
ds = pandas.DataFrame(data = StandardScaler().fit_transform(ds), index = ds.index, columns = ds.columns)

subject_ids = np.array(ds.index)

metadata = pandas.read_csv('../data/gse73072_metadata.csv', index_col = 0)
labels = metadata.loc[subject_ids]['4to2_9_16_test']
labels = np.array(labels != 'control')

ds.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
SampleID,,,,,,,,,,,,,,,,,,,,,
GSM1883444,-1.943648,-0.395996,0.338166,-1.656706,-1.138505,-1.883259,1.777784,-1.922068,-0.641514,-0.076627,...,-0.306794,-0.218273,-0.528874,-0.821310,-1.399718,-0.149125,0.291154,-0.177409,-1.538723,0.240116
GSM1883445,1.943648,0.395996,-0.338166,1.656706,1.138505,1.883259,-1.777784,1.922068,0.641514,0.076627,...,0.306794,0.218273,0.528874,0.821310,1.399718,0.149125,-0.291154,0.177409,1.538723,-0.240116
GSM1883463,-1.436357,0.061583,1.017895,-1.000333,0.681637,0.084130,1.636568,0.412668,-0.831313,-0.421505,...,-0.152611,0.087043,-0.224366,-0.320108,0.801790,0.960025,-0.464048,1.099370,-0.546082,-0.019655
GSM1883464,1.436357,-0.061583,-1.017895,1.000333,-0.681637,-0.084130,-1.636568,-0.412668,0.831313,0.421505,...,0.152611,-0.087043,0.224366,0.320108,-0.801790,-0.960025,0.464048,-1.099370,0.546082,0.019655
GSM1883482,-0.333881,-0.347623,-0.206484,0.006298,-0.767126,-0.509758,-0.017498,0.130597,-0.056278,-0.387759,...,-0.025402,-0.100490,-0.220739,-0.133690,0.395917,0.655182,-0.079819,0.904128,0.239293,0.864048


## Load the features from one pathway

In [3]:
pw = pandas.read_csv('../data/pw_edge_mtx/pw_mtx_R-HSA-9018519.csv', index_col = 0)
# pw = pandas.read_csv('../data/pw_edge_mtx/pw_mtx_R-HSA-2672351.csv', index_col = 0)

featureset = np.array(pw.index)

featureset_ds = ds[featureset]

featureset_data = np.array(featureset_ds)

## Run spectral clustering

* similarity: correlation, zobs, heat_kernel
* loo: True indicates a leave one out support vector machine experiment
* fiedler: True indicates using the laplacian and false indicates usage of the normalized laplacian 

In [4]:
my_sc = SpectralClustering(similarity = 'correlation', loo = True, fiedler = True)
sc_pipeline  = Pipeline([('scaling', StandardScaler()), ('sc', my_sc)])

In [5]:
sc_pipeline.fit(featureset_data, labels)
clst_nodes, clst_bsrs, clst_mean_edges, clst_tree = sc_pipeline.predict(X = featureset_data, y = labels)

leaf 0.22543644319860234


## Summarize the clustering


In [6]:
cluster_features = featureset[clst_nodes][0]

initial_bsr = sc_pipeline['sc'].test_cut_loo(data = featureset_data, labels = labels)

print(f'There are {len(featureset)} features in the pathway and only {len(cluster_features)} features in the detected cluster.')
print('.')
print('.')
print('.')
print('There leave one out support vector machine balanced success rates:')
print(f'In the pathway: {round(initial_bsr,2)}')
print(f'In the cluster: {[round(c,2) for c in clst_bsrs]}')
print('.')
print('.')
print('.')
print('Mean edge weight at each iteration of the clustering. (Initial mean edge weight is 0)')
print(clst_tree)


There are 242 features in the pathway and only 138 features in the detected cluster.
.
.
.
There leave one out support vector machine balanced success rates:
In the pathway: 0.89
In the cluster: [0.9]
.
.
.
Mean edge weight at each iteration of the clustering. (Initial mean edge weight is 0)

    ________0.20662910946381577
   /
0.212__
       \
      0.225



** This method can calculate more than one cluster by going down more than one path in the hierarchical clustering tree **